# HELIO-OPS Guardian System (HOGS)
### Solar Activity Tracker and Satellite Protection Console

This notebook implements a console-based Python application that:

- Records solar events (e.g., solar flares, CMEs, sunspots)
- Registers satellites in different orbits with shielding and optional hardening
- Assesses the risk from solar events to satellites
- Suggests protective actions such as entering safe mode
- Supports manual data entry, auto-generation of data, and JSON-based persistence

The code is organised into sections:
1. Imports and global state
2. Data model classes (`SolarEvent`, `Satellite`, `HardenedSatellite`)
3. Persistence helpers (save/load)
4. User input functions (add/list events and satellites)
5. Auto-generation helpers
6. Dashboards and analysis functions
7. Risk assessment tools
8. Safe mode control
9. Main menu and program loop

















In [1]:
# --- Imports and global data structures ---

import json    # for saving/loading data as JSON
import os      # for checking if the save file exists
import random  # for auto-generating random events and satellites

# Lists used as central data storage structures
solar_events = []
satellites = []

# File name for JSON persistence
DATA_FILE = "hogs_data.json"

## 1. SolarEvent Class

The `SolarEvent` class represents a single solar disturbance, such as a flare, CME, or sunspot.

Each event stores:
- `name` – identifier for the event
- `event_type` – type string (e.g., "flare", "cme")
- `magnitude` – simplified strength on a 1–10 scale
- `date` – date string in `YYYY-MM-DD` format

The class also:
- Computes a severity level (Low, Moderate, High)
- Provides a one-line summary string for display
- Supports conversion to and from dictionaries for JSON saving/loading


In [2]:
class SolarEvent:
    def __init__(self, name, event_type, magnitude, date):
        # Basic properties of a solar event
        self.name = name
        self.event_type = event_type.lower()  # normalize type to lowercase
        self.magnitude = float(magnitude)     # store as float for calculations
        self.date = date                      # keep as simple string "YYYY-MM-DD"

    def severity_level(self):
        """
        Classify the event of magnitude 1–10 as Low, Moderate, or High.
        """
        if self.magnitude >= 8:
            return "High"
        elif self.magnitude >= 4:
            return "Moderate"
        else:
            return "Low"

    def summary(self):
        """
        Return a one-line string describing this event.
        Used when listing solar events in the menu.
        """
        return (
            f"{self.name} | {self.event_type} | "
            f"Mag {self.magnitude} | {self.date} | {self.severity_level()}"
        )

    def to_dict(self):
        """
        Convert this SolarEvent into a plain dictionary.
        Suitable for JSON serialization.
        """
        return {
            "name": self.name,
            "event_type": self.event_type,
            "magnitude": self.magnitude,
            "date": self.date,
        }

    @staticmethod
    def from_dict(d):
        """
        Create a SolarEvent object back from a dictionary.
        Used when loading from JSON.
        """
        return SolarEvent(d["name"], d["event_type"], d["magnitude"], d["date"])

## 2. Satellite Classes

Two satellite-related classes are used:

1. `Satellite` (base class)
   - Models a generic satellite in LEO, MEO, or GEO.
   - Attributes:
     - `sat_id` – satellite identifier
     - `orbit_type` – "LEO", "MEO", or "GEO"
     - `shielding` – value between 0.0 and 1.0 for physical protection
     - `safe_mode` – boolean flag for current protection state
   - Methods:
     - `assess_risk()` – compute risk score and risk level for a given solar event
     - `recommended_action()` – suggest operational response
     - `info()` – one-line description
     - `to_dict()` / `from_dict()` – for JSON operations

2. `HardenedSatellite` (subclass of `Satellite`)
   - Adds:
     - `hardening` – additional factor (0.0–1.0) representing extra radiation hardening
   - Overrides:
     - `assess_risk()` to reduce the base risk using `hardening`
   - Demonstrates inheritance and polymorphism.


In [3]:
class Satellite:
    def __init__(self, sat_id, orbit_type, shielding):
        # Identifier for the satellite
        self.sat_id = sat_id
        # Normalize orbit type to upper-case (LEO/MEO/GEO)
        self.orbit_type = orbit_type.upper()
        # Shielding level between 0.0 and 1.0
        self.shielding = float(shielding)
        # Safe mode initially off
        self.safe_mode = False

    def assess_risk(self, event):
        # Pick orbit factor based on orbit type
        orbit_factor = {"LEO": 1.0, "MEO": 0.8, "GEO": 0.6}.get(self.orbit_type, 0.6)

        # Less shielding -> higher effective exposure
        protection = 1.0 - self.shielding

        # Final risk score
        risk_score = event.magnitude * orbit_factor * protection

        # Map numeric score to a textual level
        if risk_score >= 6:
            level = "High"
        elif risk_score >= 3:
            level = "Moderate"
        else:
            level = "Low"

        return risk_score, level

    def recommended_action(self, level):
        """
        Suggest an operational action based on risk level.
        """
        if level == "High":
            return "Enter SAFE MODE immediately."
        elif level == "Moderate":
            return "Reduce power and monitor closely."
        else:
            return "Normal operations."

    def info(self):
        """
        Compact one-line description of the satellite.
        """
        return (
            f"{self.sat_id} | {self.orbit_type} | "
            f"Shield {self.shielding} | SafeMode: {self.safe_mode}"
        )

    def to_dict(self):
        """
        Convert Satellite to a dictionary for JSON serialization.
        """
        return {
            "type": "Satellite",
            "sat_id": self.sat_id,
            "orbit_type": self.orbit_type,
            "shielding": self.shielding,
            "safe_mode": self.safe_mode,
        }

    @staticmethod
    def from_dict(d):
        """
        Rebuild a Satellite object from a dictionary.
        """
        sat = Satellite(d["sat_id"], d["orbit_type"], d["shielding"])
        sat.safe_mode = d.get("safe_mode", False)
        return sat


class HardenedSatellite(Satellite):
    """
    Satellite subclass with extra hardening protection.
    """

    def __init__(self, sat_id, orbit_type, shielding, hardening):
        # Initialize base Satellite attributes
        super().__init__(sat_id, orbit_type, shielding)
        # Additional hardening factor in [0.0, 1.0]
        self.hardening = float(hardening)

    def assess_risk(self, event):
        """
        Override base risk assessment.
        First, get base risk from Satellite, then reduce by hardening.
        """
        base_score, _ = super().assess_risk(event)
        # Reduce risk according to hardening factor
        adjusted = base_score * (1.0 - self.hardening)

        # Convert adjusted score into level again
        if adjusted >= 6:
            level = "High"
        elif adjusted >= 3:
            level = "Moderate"
        else:
            level = "Low"

        return adjusted, level

    def to_dict(self):
        """
        Extend Satellite's dict representation with hardening info and type.
        """
        d = super().to_dict()
        d["type"] = "HardenedSatellite"
        d["hardening"] = self.hardening
        return d

    @staticmethod
    def from_dict(d):
        """
        Rebuild a HardenedSatellite from a dictionary.
        """
        sat = HardenedSatellite(
            d["sat_id"],
            d["orbit_type"],
            d["shielding"],
            d["hardening"],
        )
        sat.safe_mode = d.get("safe_mode", False)
        return sat

## 3. Data Persistence (Save and Load)

The system supports saving and loading state using JSON.

- `save_data()`:
  - Converts all solar events and satellites to dictionaries.
  - Stores them in a JSON file.

- `load_data()`:
  - Reads JSON if the file exists.
  - Clears current in-memory lists.
  - Reconstructs `SolarEvent`, `Satellite`, and `HardenedSatellite` objects.


In [4]:
def save_data(filename=DATA_FILE):
    """
    Save all solar events and satellites to a JSON file.
    """
    data = {
        "events": [e.to_dict() for e in solar_events],
        "sats": [s.to_dict() for s in satellites],
    }
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)
    print("Data saved.")


def load_data(filename=DATA_FILE):
    """
    Load solar events and satellites from a JSON file, if it exists.
    """
    if not os.path.exists(filename):
        print("No saved data found.")
        return

    with open(filename, "r") as f:
        data = json.load(f)

    # Clear existing data to avoid duplicates
    solar_events.clear()
    satellites.clear()

    # Rebuild event objects
    for ev in data.get("events", []):
        solar_events.append(SolarEvent.from_dict(ev))

    # Rebuild satellite objects, distinguishing types
    for sd in data.get("sats", []):
        if sd.get("type") == "HardenedSatellite":
            satellites.append(HardenedSatellite.from_dict(sd))
        else:
            satellites.append(Satellite.from_dict(sd))

    print("Data loaded.")

## 4. User Input for Solar Events

These functions allow the user to:

- Add a new solar event via console input.
- List all recorded solar events in a readable format.



In [5]:
def add_solar_event():
    """
    Ask the user for solar event details and store a new SolarEvent.
    """
    print("\n--- Add Solar Event ---")
    try:
        name = input("Event name: ").strip()
        event_type = input("Type (flare/cme/sunspot): ").strip().lower()
        magnitude = float(input("Magnitude (1–10): ").strip())
        date = input("Date (YYYY-MM-DD): ").strip()

        solar_events.append(SolarEvent(name, event_type, magnitude, date))
        print("Solar event added.\n")
    except Exception:
        # Simple catch-all to keep the UI flowing
        print("Invalid input. Event not added.\n")


def list_solar_events():
    """
    Print all solar events currently stored.
    """
    print("\n--- Solar Events ---")
    if not solar_events:
        print("No events recorded.\n")
        return

    for idx, ev in enumerate(solar_events, start=1):
        print(f"{idx}. {ev.summary()}")
    print()

## 5. User Input for Satellites

These functions allow the user to:

- Add a new satellite (normal or hardened) via console input.
- List all registered satellites with their properties.


In [6]:
def add_satellite():
    """
    Ask the user for satellite details and create a Satellite or HardenedSatellite.
    """
    print("\n--- Add Satellite ---")
    try:
        sat_id = input("Satellite ID: ").strip()
        orbit = input("Orbit (LEO/MEO/GEO): ").strip().upper()
        shielding = float(input("Shielding 0.0–1.0: ").strip())

        special = input("Hardened satellite? (y/n): ").strip().lower()
        if special == "y":
            hard = float(input("Hardening factor 0.0–1.0: ").strip())
            satellites.append(HardenedSatellite(sat_id, orbit, shielding, hard))
        else:
            satellites.append(Satellite(sat_id, orbit, shielding))

        print("Satellite added.\n")
    except Exception:
        print("Invalid input. Satellite not added.\n")


def list_satellites():
    """
    Print all satellites currently stored.
    """
    print("\n--- Satellites ---")
    if not satellites:
        print("No satellites registered.\n")
        return

    for idx, sat in enumerate(satellites, start=1):
        print(f"{idx}. {sat.info()}")
    print()

## 6. Auto-Generation of Events and Satellites

Auto-generation helps create test data quickly:

- `auto_generate_events()` creates random solar events.
- `auto_generate_satellites()` creates random satellites, some of which are hardened.


In [7]:
def auto_generate_events(n=3):
    """
    Automatically create n random solar events.
    """
    types = ["flare", "cme", "sunspot"]
    for _ in range(n):
        name = f"Auto-{random.randint(100, 999)}"
        event_type = random.choice(types)
        magnitude = round(random.uniform(1, 10), 1)  # random magnitude 1.0–10.0
        date = f"2025-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}"
        solar_events.append(SolarEvent(name, event_type, magnitude, date))

    print(f"{n} solar events generated.\n")


def auto_generate_satellites(n=3):
    """
    Automatically create n satellites, some hardened.
    """
    orbits = ["LEO", "MEO", "GEO"]
    for _ in range(n):
        sat_id = f"SAT-{random.randint(100, 999)}"
        orbit = random.choice(orbits)
        shielding = round(random.uniform(0.1, 0.9), 2)  # 0.10–0.90

        # 50% chance to be hardened
        if random.random() > 0.5:
            hard = round(random.uniform(0.1, 0.6), 2)
            satellites.append(HardenedSatellite(sat_id, orbit, shielding, hard))
        else:
            satellites.append(Satellite(sat_id, orbit, shielding))

    print(f"{n} satellites generated.\n")

## 7. Dashboards and Health Check

Two summary-style functions:

- `space_weather_dashboard()` – shows basic statistics about all recorded solar events.
- `satellite_health_check()` – shows how each satellite is affected by the strongest solar event.


In [8]:
def space_weather_dashboard():
    """
    Display a summary of the current solar event dataset.
    """
    print("\n--- SPACE WEATHER DASHBOARD ---")
    print(f"Total events: {len(solar_events)}")

    if solar_events:
        avg_mag = sum(e.magnitude for e in solar_events) / len(solar_events)
        print(f"Average magnitude: {avg_mag:.2f}")

        # Count events of each type
        types = {}
        for e in solar_events:
            types[e.event_type] = types.get(e.event_type, 0) + 1
        print("Event type counts:", types)
    print()


def satellite_health_check():
    """
    Show each satellite's risk with respect to the strongest event.
    """
    print("\n--- SATELLITE HEALTH CHECK ---")
    if not solar_events:
        print("No solar events available.\n")
        return
    if not satellites:
        print("No satellites available.\n")
        return

    # Select the event with highest magnitude
    strongest = max(solar_events, key=lambda e: e.magnitude)
    print(f"Strongest event: {strongest.summary()}\n")

    for sat in satellites:
        print(sat.info())
        score, level = sat.assess_risk(strongest)
        print(f"  Risk level: {level}")
        print(f"  Recommended action: {sat.recommended_action(level)}\n")

## 8. Risk Assessment Tools

Two functions are provided:

- `assess_risks_for_event()` – choose a single solar event and check all satellites.
- `assess_risks_for_all_events()` – assess all satellites against every solar event.



In [9]:
def assess_risks_for_event():
    """
    Let the user pick one solar event and assess all satellites against it.
    """
    if not solar_events:
        print("No solar events available.\n")
        return
    if not satellites:
        print("No satellites available.\n")
        return

    list_solar_events()
    try:
        choice = int(input("Select event number: ").strip())
        event = solar_events[choice - 1]
    except Exception:
        print("Invalid selection.\n")
        return

    print(f"\n--- Risk assessment for {event.summary()} ---")
    for sat in satellites:
        score, level = sat.assess_risk(event)
        action = sat.recommended_action(level)
        print(f"{sat.sat_id} → Risk={score:.2f}, Level={level}, Action={action}")
    print()


def assess_risks_for_all_events():
    """
    Assess risk for all satellites versus all recorded events.
    """
    if not solar_events:
        print("No solar events available.\n")
        return
    if not satellites:
        print("No satellites available.\n")
        return

    print("\n--- ALL EVENTS vs ALL SATELLITES ---")
    for event in solar_events:
        print(f"\nEvent: {event.summary()}")
        for sat in satellites:
            score, level = sat.assess_risk(event)
            print(f"  {sat.sat_id} → Risk={score:.2f}, Level={level}")
    print()

## 9. Safe Mode Control

The following function allows selecting a satellite and toggling its safe mode status based on user input.


In [10]:
def toggle_satellite_safe_mode():
    """
    Let the user select a satellite and turn its safe mode ON or OFF.
    """
    if not satellites:
        print("No satellites available.\n")
        return

    list_satellites()
    try:
        choice = int(input("Select satellite number: ").strip())
        sat = satellites[choice - 1]
    except Exception:
        print("Invalid selection.\n")
        return

    print(f"Current status: {sat.info()}")
    mode = input("Turn safe mode ON? (y/n): ").strip().lower()
    sat.safe_mode = (mode == "y")
    print(f"Updated status: {sat.info()}\n")

## 10. Main Menu and Program Loop

The menu system provides a simple text-based interface:

- Displays options
- Reads user choice
- Calls the appropriate function
- Allows exiting the program cleanly


In [ ]:
def show_menu():
    """
    Print the main menu options for the HELIO-OPS Guardian System.
    """
    print("""
=== HELIO-OPS Guardian System (HOGS) ===
1. List solar events
2. Add solar event
3. Auto-generate solar events
4. List satellites
5. Add satellite
6. Auto-generate satellites
7. Assess risk for one event
8. Assess risk for all events
9. Space weather dashboard
10. Satellite health check
11. Save data
12. Load data
13. Toggle satellite safe mode
0. Exit
""")


def main():
    """
    Main loop for the console-based interface.
    """
    while True:
        show_menu()
        ch = input("Enter choice: ").strip()

        if ch == "1":
            list_solar_events()
        elif ch == "2":
            add_solar_event()
        elif ch == "3":
            auto_generate_events()
        elif ch == "4":
            list_satellites()
        elif ch == "5":
            add_satellite()
        elif ch == "6":
            auto_generate_satellites()
        elif ch == "7":
            assess_risks_for_event()
        elif ch == "8":
            assess_risks_for_all_events()
        elif ch == "9":
            space_weather_dashboard()
        elif ch == "10":
            satellite_health_check()
        elif ch == "11":
            save_data()
        elif ch == "12":
            load_data()
        elif ch == "13":
            toggle_satellite_safe_mode()
        elif ch == "0":
            print("Exiting HELIO-OPS Guardian System.")
            break
        else:
            print("Invalid choice.\n")


# Run the system if this cell is executed as the main script context
if __name__ == "__main__":
    main()


=== HELIO-OPS Guardian System (HOGS) ===
1. List solar events
2. Add solar event
3. Auto-generate solar events
4. List satellites
5. Add satellite
6. Auto-generate satellites
7. Assess risk for one event
8. Assess risk for all events
9. Space weather dashboard
10. Satellite health check
11. Save data
12. Load data
13. Toggle satellite safe mode
0. Exit

Enter choice: 1

--- Solar Events ---
No events recorded.


=== HELIO-OPS Guardian System (HOGS) ===
1. List solar events
2. Add solar event
3. Auto-generate solar events
4. List satellites
5. Add satellite
6. Auto-generate satellites
7. Assess risk for one event
8. Assess risk for all events
9. Space weather dashboard
10. Satellite health check
11. Save data
12. Load data
13. Toggle satellite safe mode
0. Exit

Enter choice: 2

--- Add Solar Event ---
Event name: Helo-56
Type (flare/cme/sunspot): sunspot
Magnitude (1–10): 0.5
Date (YYYY-MM-DD): 2020-12-12
Solar event added.


=== HELIO-OPS Guardian System (HOGS) ===
1. List solar even